# How to Detect Light Bars

In [2]:
import cv2
import numpy as np
import imutils
import math
from operator import itemgetter
print(cv2.__version__)

3.4.0


## Declare the Source

In [2]:
cap = cv2.VideoCapture('test.mp4') # Set to integer starting at 0 to use peripheral cameras

if not cap.isOpened():
    print('Error opening video stream or file')

Error opening video stream or file


## Play the Video

In [9]:
cap = cv2.VideoCapture('../../test.mp4')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        
        cv2.imshow('Frame', frame)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break

cap.release()
cv2.destroyAllWindows()

## Convert To HSV

### What's HSV?

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/HSV_color_solid_cylinder_alpha_lowgamma.png/800px-HSV_color_solid_cylinder_alpha_lowgamma.png)

In [10]:
cap = cv2.VideoCapture('../../test.mp4')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        cv2.imshow('HSV', hsv)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Detect Lights From HSV

In [11]:
#cap = cv2.VideoCapture('../../RedCar.avi')
cap = cv2.VideoCapture('../../test.mp4')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,15,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        cv2.imshow('Lights', lights)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Convert Back to BGR

Convert back to BGR for further manipulation

In [238]:
cap = cv2.VideoCapture('RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,3,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        lights_bgr = cv2.cvtColor(lights, cv2.COLOR_HSV2BGR)
        
        cv2.imshow('BGR Lights', lights)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Blur

In [10]:
cap = cv2.VideoCapture('../../RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,3,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        lights_bgr = cv2.cvtColor(lights, cv2.COLOR_HSV2BGR)
        blurred = cv2.GaussianBlur(lights_bgr, (5,5), 0)    
        
        cv2.imshow('Blurred', blurred)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Threshold

In [8]:
cap = cv2.VideoCapture('../../test.mp4')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,3,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        lights_bgr = cv2.cvtColor(lights, cv2.COLOR_HSV2BGR)
        blurred = cv2.GaussianBlur(lights_bgr, (5,5), 0)
        
        thresh = cv2.threshold(blurred, 50, 255, cv2.THRESH_BINARY)[1]
        
        
        
        cv2.imshow('Threshold', thresh)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Define Detect Shapes Function

In [3]:
def detect(c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        # if the shape is a triangle, it will have 3 vertices
        if len(approx) == 3:
            shape = "triangle"
 
        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
 
            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            if ar >= 0.95 and ar <= 1.05:
                shape = "square"
            elif ar <= 0.50:
                shape = "lightbar"
            else:
                shape = "rectangle"
                 
        # if the shape is a pentagon, it will have 5 vertices
        elif len(approx) == 5:
            shape = "pentagon"
 
        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"
 
        # return the name of the shape
        return (shape, approx)

## Draw Shapes on Threshold

In [4]:
cap = cv2.VideoCapture('../../RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,3,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        lights_bgr = cv2.cvtColor(lights, cv2.COLOR_HSV2BGR)
        blurred = cv2.GaussianBlur(lights_bgr, (5,5), 0)
        
        thresh = cv2.threshold(blurred, 30, 255, cv2.THRESH_BINARY)[1]
        
        gray = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
        
        
        cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        
        cnts = cnts[0] if imutils.is_cv2() else cnts[1]
        
        
        for c in cnts:
            try:
                # compute the center of the contour, then detect the name of the
                # shape using only the contour
                M = cv2.moments(c)
                cX = int((M["m10"] / M["m00"]))
                cY = int((M["m01"] / M["m00"]))
                
                shape, approx = detect(c)


                if shape == 'lightbar':
                    cv2.drawContours(thresh, [c], -1, (0, 255, 0), 2)
                    cv2.putText(thresh, 'lightbar', (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    #draw bisecting lines
                    (x, y, w, h) = cv2.boundingRect(approx)
                    cv2.line(thresh, (int((x+(w/2))), y), (int((x+(w/2))), y+h), (255,0,0), 2)


            except:
                pass
        
        
        
        cv2.imshow('Threshold', thresh)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Draw Shapes on Original Frame

In [5]:
cap = cv2.VideoCapture('../../RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,3,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        lights_bgr = cv2.cvtColor(lights, cv2.COLOR_HSV2BGR)
        blurred = cv2.GaussianBlur(lights_bgr, (5,5), 0)
        
        thresh = cv2.threshold(blurred, 30, 255, cv2.THRESH_BINARY)[1]
        
        gray = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
        
        
        cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        
        cnts = cnts[0] if imutils.is_cv2() else cnts[1]
        
        prev_coords = None
        
        
        for c in cnts:
            try:
                # compute the center of the contour, then detect the name of the
                # shape using only the contour
                #M = cv2.moments(c)
                #cX = int((M["m10"] / M["m00"]))
                #cY = int((M["m01"] / M["m00"]))
                
                shape, approx = detect(c)


                if shape == 'lightbar':
                    cv2.drawContours(frame, [c], -1, (0, 255, 0), 2)
                    #cv2.putText(frame, 'lightbar', (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    #draw bisecting lines
                    (x, y, w, h) = cv2.boundingRect(approx)
                    cv2.line(frame, (int((x+(w/2))), y), (int((x+(w/2))), y+h), (255,0,0), 2)


            except:
                pass
        
        
        
        cv2.imshow('Output', frame)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Draw Box Around Target

In [6]:
def detect(c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        # if the shape is a triangle, it will have 3 vertices
        if len(approx) == 3:
            shape = "triangle"
 
        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
 
            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            if ar >= 0.95 and ar <= 1.05:
                shape = "square"
            elif ar <= 0.50:
                shape = "lightbar"
            else:
                shape = "rectangle"
                 
        # if the shape is a pentagon, it will have 5 vertices
        elif len(approx) == 5:
            shape = "pentagon"
 
        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"
 
        # return the name of the shape
        return (shape, approx)

In [7]:
"""
@param frames Number of frames to store in history.
"""
class FrameHist:
    
    def __init__(self, frames):
        self.frames = frames
        self.data = []
        
    #adds data to data list
    #deletes last item in list if it is longer than the defined history
    def update(self, data=None):
        if data:
            self.data.append(data)
            
        if len(self.data) > self.frames:
            del self.data[-1]    

In [8]:
def distance(p0, p1):
    return math.sqrt((p0[0] - p1[0])**2 + (p0[1] - p1[1])**2)

In [12]:
#cap = cv2.VideoCapture('../../test.mp4')
cap = cv2.VideoCapture('../../RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,20,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        lights_bgr = cv2.cvtColor(lights, cv2.COLOR_HSV2BGR)
        blurred = cv2.GaussianBlur(lights_bgr, (5,5), 0)
        
        thresh = cv2.threshold(blurred, 30, 255, cv2.THRESH_BINARY)[1]
        
        gray = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
        
        
        cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        
        cnts = cnts[0] if imutils.is_cv2() else cnts[1]
        
        prev_coords = None
        
        lightbars = []
        
        for c in cnts:
            try:
                
                shape, approx = detect(c)


                if shape == 'lightbar':
                    cv2.drawContours(frame, [c], -1, (0, 255, 0), 2)
                    #cv2.putText(frame, 'lightbar', (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    #draw bisecting lines
                    (x, y, w, h) = cv2.boundingRect(approx)
                    lightbar_top = (int((x+(w/2))), y)
                    lightbar_bot = (int((x+(w/2))), y+h)
                    lightbar_len = distance(lightbar_top, lightbar_bot)
                    
                    target_top = (lightbar_top[0], lightbar_top[1] - int(lightbar_len/1.7))
                    target_bot = (lightbar_bot[0], lightbar_bot[1] + int(lightbar_len/1.7))
                    target_len = distance(target_top, target_bot)

                    lightbars.append((target_top, target_bot, target_len))
                    #print(lightbar_len)
                    
                    #line bisecting the lightbar retangle
                    cv2.line(frame, lightbar_top, lightbar_bot, (255,0,0), 2)
                
                    
                    #line besecting the lightbar rectangle that borders the target box (side)
                    #cv2.line(frame,target_bot,target_top,(0,0,255),2)


            except:
                pass
            
        #for lightbar in lightbars:
            
        #    print(lightbar[2])
        
        lightbars = sorted(lightbars, key=itemgetter(2))
        
        #for i in range(len(lightbars)):
        #    diff = lightbars[i+1][2] - lightbars[i][2] 
            
        
        if len(lightbars) == 2:
            cv2.line(frame, lightbars[0][0], lightbars[1][0], (0,0,255), 2)
            cv2.line(frame, lightbars[0][1], lightbars[1][1], (0,0,255), 2)
            cv2.line(frame, lightbars[0][0], lightbars[0][1], (0,0,255), 2)
            cv2.line(frame, lightbars[1][0], lightbars[1][1], (0,0,255), 2)
            
            
            print(lightbars)
        #print('----------')
        
        
        cv2.imshow('Output', frame)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

[((436, 264), (436, 453), 189.0), ((218, 238), (218, 428), 190.0)]
[((436, 262), (436, 451), 189.0), ((218, 237), (218, 427), 190.0)]
[((437, 257), (437, 446), 189.0), ((218, 230), (218, 423), 193.0)]
[((437, 245), (437, 435), 190.0), ((222, 220), (222, 413), 193.0)]
[((436, 244), (436, 433), 189.0), ((220, 220), (220, 410), 190.0)]
[((437, 241), (437, 431), 190.0), ((219, 218), (219, 408), 190.0)]
[((436, 238), (436, 428), 190.0), ((220, 214), (220, 407), 193.0)]
[((437, 236), (437, 426), 190.0), ((220, 211), (220, 405), 194.0)]
[((436, 236), (436, 426), 190.0), ((219, 212), (219, 405), 193.0)]
[((435, 235), (435, 425), 190.0), ((218, 211), (218, 405), 194.0)]
[((434, 234), (434, 424), 190.0), ((216, 210), (216, 403), 193.0)]
[((433, 233), (433, 423), 190.0), ((215, 210), (215, 403), 193.0)]
[((198, 193), (198, 383), 190.0), ((416, 210), (416, 403), 193.0)]
[((405, 203), (405, 392), 189.0), ((191, 191), (191, 380), 189.0)]
[((401, 201), (401, 390), 189.0), ((187, 187), (187, 380), 193